In [3]:
using PortAudio

In [2]:
using Pkg; Pkg.activate("../../../DynSysAudio")

In [4]:
function get_soundcard_matching(;output::String="Speaker",host::String="MME",nchannels::Int=2)
    a = PortAudio.devices()
    for d in a
        if occursin(output, d.name) && occursin(host, d.host_api)
            name = d.name
            println("Using device: $name")
            return PortAudioStream(d, 0, nchannels)
        end   
    end
    print("no match")
end

get_soundcard_matching (generic function with 1 method)

In [16]:
sdev = PortAudio.devices()

26-element Vector{PortAudio.PortAudioDevice}:
 "Microsoft Sound Mapper - Input" 2→0
 "Microphone (Logitech BRIO)" 2→0
 "Microphone (Realtek(R) Audio)" 2→0
 "Microsoft Sound Mapper - Output" 0→2
 "Speakers (Realtek(R) Audio)" 0→8
 "Nx Headphones (Waves Virtual Au" 0→8
 "Realtek Digital Output (Realtek" 0→2
 "Primary Sound Capture Driver" 2→0
 "Microphone (Logitech BRIO)" 2→0
 "Microphone (Realtek(R) Audio)" 2→0
 "Primary Sound Driver" 0→2
 "Speakers (Realtek(R) Audio)" 0→8
 "Nx Headphones (Waves Virtual Audio Device)" 0→8
 "Realtek Digital Output (Realtek(R) Audio)" 0→2
 "Nx Headphones (Waves Virtual Audio Device)" 0→8
 "Speakers (Realtek(R) Audio)" 0→2
 "Realtek Digital Output (Realtek(R) Audio)" 0→2
 "Microphone (Realtek(R) Audio)" 2→0
 "Microphone (Logitech BRIO)" 2→0
 "Speakers (Realtek HD Audio output)" 0→8
 "Stereo Mix (Realtek HD Audio Stereo input)" 2→0
 "Line In (Realtek HD Audio Line input)" 2→0
 "Microphone (Realtek HD Audio Mic input)" 2→0
 "SPDIF Out (Realtek HDA SPDIF Out)

In [13]:
soundcard = get_soundcard_matching(nchannels=2)

Using device: Speakers (Realtek(R) Audio)


PortAudioStream{Float32}
  Samplerate: 44100.0Hz
  2 channel sink: "Speakers (Realtek(R) Audio)"

In [6]:
using SampledSignals, Unitful

In [8]:
using DynSysAudio 

In [9]:
using Pipe: @pipe

In [10]:
tone_source = SinusoidSource(Float64, 44100, [200, 310])

SinusoidSource{Float64}(44100.0, [0.02849517146113191, 0.04416751576475446], [0.0, 0.0])

In [11]:
tone_source.freqs

2-element Vector{Float64}:
 0.02849517146113191
 0.04416751576475446

In [15]:
noise_stream = Threads.@spawn begin
    while tone_source.freqs[1] > 0.001
        @pipe read(tone_source, 0.1u"s") |> write(soundcard, _)
    end
end

Task (runnable) @0x000000008c44cf90

In [17]:
tone_source.freqs[1] = 300

300

In [ ]:
s0 = read(tone_source, 0.5u"s")

In [ ]:
using Plots
plot(domain(ss), ss)

In [ ]:
noise_source = NoiseSource(Float64, 44100, 2, 0.2)

In [ ]:
function vdp!(du,u,p,t)
    du[1] = u[2]
    du[2] = p[1]*(1.0-u[1]*u[1])*u[2]-u[1]
end	

In [13]:
function halvorsen!(du,u,p,t)
    du[1] = -p[1]*u[1]-4.0*u[2]-4.0*u[3]-u[2]*u[2]
    du[2] = -p[1]*u[2]-4.0*u[3]-4.0*u[1]-u[3]*u[3]
    du[3] = -p[1]*u[3]-4.0*u[1]-4.0*u[2]-u[1]*u[1]
end    

halvorsen! (generic function with 1 method)

In [44]:
ode_source2 = ODESource(Float64, halvorsen!, 44100, 200, [1.0, 0.1,0.1],[1.4])

ODESource{Float64}(44100.0, 3, 0.0, 0.0045351473922902496, 0.1, SciMLBase.ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, Vector{Float64}, SciMLBase.ODEFunction{true, typeof(halvorsen!), LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing}, Base.Iterators.Pairs{Union{}, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}, SciMLBase.StandardODEProblem}(SciMLBase.ODEFunction{true, typeof(halvorsen!), LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing}(halvorsen!, LinearAlgebra.UniformScaling{Bool}(true), nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, SciMLBase.DEFAULT_OBSERVED, nothing), [1.0, 0.1, 0.1], (0.0, 100.0), [1.4], Base.Iterators.Pairs{Union{}, Uni

In [ ]:
ss = read(ode_source, 1.0u"s")

In [ ]:
using Plots
plot(domain(ss), ss)

In [37]:
ode_stream = Threads.@spawn begin
    while ode_source.integrator.p[1] > 0
        @pipe read(ode_source, 0.5u"s") |> write(soundcard, _)
    end
end

Task (runnable) @0x000000009cd57d00

In [ ]:
ode_source.integrator.p[1] = 1.4

In [49]:
ode_source.dt = 90/44100.0

0.0020408163265306124

In [45]:
ode_stream2 = Threads.@spawn begin
    while ode_source2.integrator.p[1] > 0
        @pipe read(ode_source2, 0.5u"s") |> write(soundcard, _)
    end
end

Task (runnable) @0x000000000e4b01a0

In [46]:
ode_source2.integrator.p[1] = 0

0

In [ ]:
using DifferentialEquations

In [ ]:
problem = ODEProblem(halvorsen!,[1.0, 0.1,0.1],(0.0,600),[1.4])

In [ ]:
integrator = init(problem,Tsit5())

In [ ]:
seq = TimeChoiceIterator(integrator,0.0:200/44100.0:200.0)

In [ ]:
X = reduce(vcat,[[u[1] u[2] u[3]] for (u,t) in seq])

In [ ]:
plot(X[1,:],X[2,:])

In [ ]:
seq

In [ ]:
buf = reduce(vcat,[u[:]' for (u,t) in seq])

In [ ]:
seq